In [21]:
using CSV
using DataFrames
using JuMP
using Gurobi
using Statistics

# ---------------------------------------------
# 1. Load baseline EV registrations (non-scenario dataset)
#    cleaned_registration_ny_with_coords.csv
# ---------------------------------------------
reg_raw = CSV.read("../data/cleaned_reg_1128.csv", DataFrame)

# Make sure ZIP is an Int (often it shows up as Float64 or String)
if eltype(reg_raw.ZIP) <: AbstractFloat
    reg_raw.ZIP = round.(Int, reg_raw.ZIP)
elseif eltype(reg_raw.ZIP) <: AbstractString
    reg_raw.ZIP = parse.(Int, strip.(reg_raw.ZIP))
end

# If there are multiple rows per ZIP, aggregate vehicle_count
# (if it's already one row per ZIP, this just passes through)
reg_zip = combine(
    groupby(reg_raw, :ZIP),
    :N => sum => :vehicle_count,
)

println("Number of ZIPs in registration file: ", nrow(reg_zip))

# ---------------------------------------------
# 2. Stations (ID, Status Code, total_evse)
# ---------------------------------------------
stations = CSV.read("../data/cleaned_station.csv", DataFrame)

stations.total_evse =
    coalesce.(stations[!, "EV Level1 EVSE Num"], 0.0) .+
    coalesce.(stations[!, "EV Level2 EVSE Num"], 0.0) .+
    coalesce.(stations[!, "EV DC Fast Count"], 0.0)

stations_small = select(stations, :ID, :"Status Code", :total_evse)

# ---------------------------------------------
# 3. Distance matrix dij (ZIP × station)
#    Structure: demand_zip | 35272 | 35562 | ...
# ---------------------------------------------
dist_raw = CSV.read("../data/dij.csv", DataFrame)

# Map ZIP → row index in dist_raw
zip_to_row = Dict(dist_raw.demand_zip[r] => r for r in 1:nrow(dist_raw))

# All station columns in the distance matrix
station_cols = names(dist_raw)[2:end]

# Helper: column name -> Int station ID (works for Symbols/Strings)
station_id_from_colname(name) = parse(Int, String(name))

# Station IDs as they appear in dij.csv
station_ids_from_dist = [station_id_from_colname(c) for c in station_cols]

# Restrict to stations that appear in BOTH station table and distance matrix
ids_from_stations = Set(stations_small.ID)
candidate_station_ids = [sid for sid in station_ids_from_dist if sid in ids_from_stations]

println("Number of candidate stations (intersection): ",
        length(candidate_station_ids))

# Create J index set based on candidate stations
station_ids = candidate_station_ids
J = collect(1:length(station_ids))
nJ = length(J)

# Map station_id -> column index in dist_raw
col_idx_for_station = Dict{Int,Int}()
for (k, col_name) in enumerate(station_cols)
    sid = station_id_from_colname(col_name)
    if sid in station_ids
        col_idx_for_station[sid] = k + 1   # +1 because col 1 is :demand_zip
    end
end

# ---------------------------------------------
# 4. Align ZIPs between registration file and dij,
#    and build baseline demand vector D and distance matrix d
# ---------------------------------------------

# Keep only ZIPs that appear in BOTH reg_zip and dist_raw
valid_zips = sort(
    collect(
        intersect(Set(reg_zip.ZIP), Set(dist_raw.demand_zip))
    )
)

println("ZIPs in both registration & dij: ", length(valid_zips))

# Internal demand-node index set
zip_codes = valid_zips              # external labels for demand nodes
I = collect(1:length(zip_codes))    # internal indices 1..|I|
nI = length(I)

# Build a dictionary ZIP -> vehicle_count
veh_dict = Dict(row.ZIP => row.vehicle_count for row in eachrow(reg_zip))

# Parameter D[i]: expected daily charging demand at ZIP i
sessions_per_EV_per_day = 0.25
D = [sessions_per_EV_per_day * veh_dict[z] for z in zip_codes]

println("Total baseline EV sessions/day (sum D): ", sum(D))

# Build dense distance matrix d[i,j] (in km) aligned with I and J
d = Matrix{Float64}(undef, nI, nJ)

for (i_idx, z) in enumerate(zip_codes)
    if !haskey(zip_to_row, z)
        error("ZIP $z from registration file not found in dij.csv; ",
              "either fix dij.csv or drop this ZIP.")
    end
    row = zip_to_row[z]
    for (j_idx, sid) in enumerate(station_ids)
        col = col_idx_for_station[sid]
        d[i_idx, j_idx] = dist_raw[row, col]
    end
end

println("Built distance matrix d of size ", size(d))

Number of ZIPs in registration file: 1064
Number of candidate stations (intersection): 5594
ZIPs in both registration & dij: 1064
Total baseline EV sessions/day (sum D): 320332.0
Built distance matrix d of size (1064, 5594)


In [22]:
# ==========================================================
# 2. Global parameters from your proposal
# ==========================================================

# Capacity per charger (sessions/day)
μ_per_charger = 3.65
μ = fill(μ_per_charger, nJ)          # µ_j is homogeneous

# Fixed & variable cost parameters
F = fill(25_000.0, nJ)               # F_j = $25,000
v = fill(15_000.0, nJ)               # v_j = $15,000 per charger

# Detour + environmental cost coefficients
β = 0.7                              # $/km
α = 0.013                            # $/km
cost_per_km = β + α                  # = 0.713 $/km

# Budget and big-M
B = 1_000_000_000.0                  # $1B
M = 50.0                             # max chargers per site (tunable upper bound)

max_chargers_per_station = 10

10

In [23]:
using CSV
using DataFrames
using JuMP
using Gurobi

function solve_baseline_model(R_max_km::Float64;
                              write_outputs::Bool = false,
                              tag::String = "")

    # Penalty per unmet charging session
    λ_unmet = 15000.0   # tune as you like

    # ---------- 3.1 Feasible (i,j) pairs within R_max ----------
    edges = Tuple{Int,Int}[]
    for i in I, j in J
        if d[i,j] <= R_max_km
            push!(edges, (i,j))
        end
    end

    N_i = Dict(i => Int[] for i in I)
    N_j = Dict(j => Int[] for j in J)
    for (i,j) in edges
        push!(N_i[i], j)
        push!(N_j[j], i)
    end

    println("R_max = $(R_max_km) km → feasible (i,j) pairs: ", length(edges))

    # ---------- 3.2 Build JuMP model ----------
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 600.0)
    set_optimizer_attribute(model, "MIPGap", 0.01)

    @variable(model, y[j in J], Bin)         # open station j
    @variable(model, z[j in J] >= 0, Int)    # chargers at j
    @variable(model, x[edges] >= 0)          # fraction of demand i served by j
    @variable(model, 0 .<= u[i in I] .<= 1)  # unmet demand fraction at i

    # ---------- 3.3 Constraints ----------
    # Demand balance: served + unmet = 1
    @constraint(model, demand_balance[i in I],
        sum(x[(i,j)] for j in N_i[i]) + u[i] == 1
    )

    # Capacity
    @constraint(model, capacity[j in J],
        sum(D[i] * x[(i,j)] for i in N_j[j]) <= μ[j] * z[j]
    )

    # Assignment link
    @constraint(model, assignment_link[(i,j) in edges],
        x[(i,j)] <= y[j]
    )

    # Charger–opening link + hard cap of 10 chargers per station
    @constraint(model, charger_link[j in J],
        z[j] <= M * y[j]
    )
    @constraint(model, charger_cap[j in J],
        z[j] <= 10
    )

    # Budget
    @constraint(model, budget,
        sum(F[j] * y[j] + v[j] * z[j] for j in J) <= B
    )

    # ---------- 3.4 Objective with unmet demand ----------
    @objective(model, Min,
        # infra cost
        sum(F[j] * y[j] + v[j] * z[j] for j in J) +
        # detour + env cost
        sum(cost_per_km * d[i,j] * D[i] * x[(i,j)] for (i,j) in edges) +
        # unmet demand penalty
        λ_unmet * sum(D[i] * u[i] for i in I)
    )

    # ---------- 3.5 Solve ----------
    optimize!(model)

    term_status = termination_status(model)
    println("Termination status: ", term_status)
    obj_val = objective_value(model)
    println("Objective value:    ", obj_val)

    # ---------- 3.6 Extract and summarize ----------
    open_sites_idx = [j for j in J if value(y[j]) > 0.5]
    num_open = length(open_sites_idx)
    total_chargers = sum(value(z[j]) for j in J)

    total_fixed = sum(F[j] * value(y[j]) for j in J)
    total_var   = sum(v[j] * value(z[j]) for j in J)
    total_detour_cost =
        sum(cost_per_km * d[i,j] * D[i] * value(x[(i,j)]) for (i,j) in edges)
    total_unmet_penalty =
        λ_unmet * sum(D[i] * value(u[i]) for i in I)

    total_detour_distance =
        sum(d[i,j] * D[i] * value(x[(i,j)]) for (i,j) in edges)

    total_served = sum(D[i] * (1.0 - value(u[i])) for i in I)
    total_unmet  = sum(D[i] * value(u[i]) for i in I)
    total_demand = total_served + total_unmet
    served_fraction = total_served / max(total_demand, 1e-9)

    avg_detour = total_detour_distance / max(total_served, 1e-9)

    println("---- Baseline summary for R_max = $(R_max_km) km ----")
    println("Open stations:        ", num_open)
    println("Total chargers:       ", total_chargers)
    println("Total fixed cost:     \$", round(total_fixed;      digits=2))
    println("Total charger cost:   \$", round(total_var;        digits=2))
    println("Detour+env cost:      \$", round(total_detour_cost; digits=2))
    println("Unmet demand penalty: \$", round(total_unmet_penalty; digits=2))
    println("Objective value:      \$", round(obj_val;          digits=2))
    println("Total demand (exp.):  ", round(total_demand;       digits=2))
    println("Served demand:        ", round(total_served;       digits=2))
    println("Unmet demand:         ", round(total_unmet;        digits=2))
    println("Served fraction:      ", round(served_fraction;    digits=3))
    println("Avg detour distance:  ",
            round(avg_detour; digits=3),
            " km per served demand unit")
    println("----------------------------------------------------")

        # =====================================================
    # Build a DataFrame with ALL decisions
    # =====================================================
    decisions = DataFrame(
        model_type = String[],
        R_max      = Float64[],
        var_type   = String[],
        ZIP        = Union{Missing,Int}[],
        station_id = Union{Missing,Int}[],
        value      = Float64[],
    )

    # y_j decisions (station open)
    for j in J
        push!(decisions, (
            "baseline",         # model_type
            R_max_km,           # R_max
            "y",                # var_type
            missing,            # ZIP not applicable
            station_ids[j],     # station_id
            value(y[j])         # value
        ))
    end

    # z_j decisions (chargers at station j)
    for j in J
        push!(decisions, (
            "baseline",
            R_max_km,
            "z",
            missing,
            station_ids[j],
            value(z[j])
        ))
    end

    # u_i decisions (unmet fraction at ZIP i)
    # If you don't have u[i] in your model, you can comment this loop out.
    for (i, zcode) in enumerate(zip_codes)
        push!(decisions, (
            "baseline",
            R_max_km,
            "u",
            zcode,         # ZIP
            missing,       # station_id
            value(u[i])    # unmet fraction
        ))
    end

    # x_ij decisions (fraction of demand i served by station j)
    # To avoid a huge file, we drop very small values.
    tol = 1e-6
    for (i,j) in edges
        x_val = value(x[(i,j)])
        if x_val > tol
            push!(decisions, (
                "baseline",
                R_max_km,
                "x",
                zip_codes[i],    # ZIP
                station_ids[j],  # station_id
                x_val
            ))
        end
    end

    # (optional) keep your old chosen_stations / assignments exports too
    info_map = Dict(row.ID => row for row in eachrow(stations_small))

    chosen_stations = DataFrame(
        station_id   = Int[],
        status_code  = String[],
        total_evse   = Float64[],
        chargers_opt = Float64[]
    )

    for j in open_sites_idx
        sid = station_ids[j]
        row = info_map[sid]
        push!(chosen_stations, (
            sid,
            row."Status Code",
            row.total_evse,
            value(z[j])
        ))
    end

    assignments = DataFrame(
        ZIP         = Int[],
        station_id  = Int[],
        frac_demand = Float64[]
    )
    for (i,j) in edges
        x_val = value(x[(i,j)])
        if x_val > 1e-4
            push!(assignments, (zip_codes[i], station_ids[j], x_val))
        end
    end

    if write_outputs
        suffix = isempty(tag) ? "R$(Int(R_max_km))" : tag
        CSV.write("../data/baseline/chosen_stations_$(suffix).csv", chosen_stations)
        CSV.write("../data/baseline/assignments_$(suffix).csv", assignments)
    end

    return (
        model = model,
        decisions = decisions,
        chosen_stations = chosen_stations,
        assignments = assignments,
        summary = (
            R_max = R_max_km,
            open_sites = num_open,
            total_chargers = total_chargers,
            total_fixed = total_fixed,
            total_var = total_var,
            total_detour_cost = total_detour_cost,
            total_unmet_penalty = total_unmet_penalty,
            total_demand = total_demand,
            served_demand = total_served,
            unmet_demand = total_unmet,
            served_fraction = served_fraction,
            avg_detour_km = avg_detour,
            objective = obj_val,
            term_status = term_status
        )
    )
end

solve_baseline_model (generic function with 1 method)

In [24]:
Rmax_values = [15.0]
results = Dict{Float64,Any}()

for R in Rmax_values
    println("\n=== Solving baseline model for R_max = $R km ===")
    res = solve_baseline_model(R; write_outputs = true)
    results[R] = res
end


=== Solving baseline model for R_max = 15.0 km ===
R_max = 15.0 km → feasible (i,j) pairs: 186729
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
TimeLimit  600
MIPGap  0.01

Optimize a model with 204576 rows, 198981 columns and 781544 nonzeros
Model fingerprint: 0x72386258
Variable types: 187793 continuous, 11188 integer (5594 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+04]
  Objective range  [2e-02, 6e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+09]
Found heuristic solution: objective 4.804980e+09
Presolve removed 6390 rows and 885 columns (presolve